In [24]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [25]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [26]:
X_train =  np.load('/content/gdrive/My Drive/Assignment 5/X_train.npy', allow_pickle=True)
y_train =  np.load('/content/gdrive/My Drive/Assignment 5/y_train.npy', allow_pickle=True)
X_test =  np.load('/content/gdrive/My Drive/Assignment 5/X_test.npy' ,allow_pickle=True)

In [27]:
df_Xtrain = pd.DataFrame(X_train,columns=['reviewerID','reviewText','summary'])
df_ytrain = pd.DataFrame(y_train,columns=['overall'])
df_train = pd.concat([df_ytrain, df_Xtrain], axis=1)
df_Xtest = pd.DataFrame(X_test,columns=['reviewerID','reviewText','summary'])

In [28]:
print(f'First review = {df_train.loc[0, "reviewText"]}')
print(f'First review has length = {len(df_train.loc[0, "reviewText"])}\n ')
print(f'First review summary= {df_train.loc[0, "summary"]}')
print(f'First review summary has length = {len(df_train.loc[0, "summary"])}\n ')
print(f'First review overall rating = {df_train.loc[0, "overall"]}')

First review = One of my favorite perfumes and the fact that it is unisex is awesome. I'm gifting this for my nephew.
First review has length = 102
 
First review summary= One of my favorite perfumes and the fact that it is unisex is ...
First review summary has length = 65
 
First review overall rating = 4


Making the textual data ready for the RNN model is a bit more involved than for the previous models. We will use the Keras Tokenizer to do most of the work for us. The Tokenizer will split the text into words for us, and create a vocabulary with an index number for each word. We can then represent each text sample by the index numbers of the words in the text. See lecture notes for more details.

**max_tokens**: So in this case we want the vocabulary to consist of the 1000 most frequent words.

**output_sequence_length**: So we do not want sequences of more than length 100 (100 words).

**pad_to_max_tokens = True:** So we want to padd with 0's - such that we will have equal length sequences (100 words).

**df_train:** This line subtracts 1 from the "overall" column in the DataFrame df_train. It appears to be converting the labels from a 1-5 scale to a 0-4 scale.

**df_train:** This line fills any NaN (missing) values in the "reviewText" column with an empty string ('') and ensures that the column's data type is converted to string (str).

**encoder:** Here, a TextVectorization layer is initialized with the previously defined max_tokens, output_sequence_length, and pad_to_max_tokens parameters. This layer will be used to vectorize the text data.

So if i have a sentence as a input then this will create a layer, where each word of the sentence - if one of the 1000 most frequent - will be a vector?

**text_ds**: A TensorFlow Dataset is created from the "reviewText" column of the DataFrame df_train. It is batched into groups of 128 samples. This dataset will be used to adapt the TextVectorization layer to the text data.

**encoder.adapt:** : The TextVectorization layer is adapted to the text dataset text_ds. This step helps the TextVectorization layer build its vocabulary based on the text data and prepares it for vectorization.

**train_ds:** A new TensorFlow Dataset is created from the "reviewText" and "overall" columns of df_train. The data is batched into groups of 128 samples. This dataset combines the text data and labels.

**lambda:** The TextVectorization layer is applied to the text data in the dataset train_ds. It maps the text sequences to their corresponding integer representations using the vocabulary created earlier.

Overall, this line of code is transforming your train_ds dataset so that it now contains pairs of vectorized text data and their corresponding labels. This transformed dataset can be used as input to a machine learning model for training, where the text data is represented in a numerical format suitable for modeling, and the labels are used for supervised learning

**AUTOTUNE:** This line defines the AUTOTUNE constant, which is used to optimize dataset performance by allowing TensorFlow to automatically adjust the number of parallel data loading operations based on available CPU resources.

**Last line:** The dataset is cached and prefetched for better performance. Caching stores elements in memory after the first epoch, and prefetching overlaps data preprocessing with model training, reducing training time. buffer_size specifies how many elements to prefetch.

**Setup Parameters:**

max_tokens: Limits the vocabulary size based on word frequency.
output_sequence_length: Sets a maximum sequence length.
pad_to_max_tokens: Determines whether sequences should be padded to the specified maximum sequence length.
Other data preprocessing parameters are defined.

**Data Preprocessing:**

Convert labels in the "overall" column of the DataFrame to a 0-4 scale.
Fill any missing values in the "reviewText" column with empty strings and convert the column to strings.¨

**TextVectorization Layer Initialization:**

Initialize a TextVectorization layer with the defined parameters. This layer will be used to vectorize the text data.

**Adapt TextVectorization Layer:**

Create a dataset text_ds from the "reviewText" column of the DataFrame and batch it for processing.
Adapt the TextVectorization layer to the text dataset text_ds, building a vocabulary based on the text data.
Retrieve the vocabulary created by the TextVectorization layer.

**Create Training Dataset:**

Create a dataset train_ds from the "reviewText" and "overall" columns of the DataFrame and batch it for processing.
Apply the TextVectorization layer to the text data in the dataset, converting text sequences to numerical representations using the vocabulary.

**Dataset Configuration for Performance:**

Define AUTOTUNE for optimizing dataset performance.
Cache the training dataset for faster access.
Prefetch data for efficient data loading during model training.
Overall, this code snippet prepares text data for machine learning by converting it into a format suitable for neural network models. It uses the TextVectorization layer to transform text into numerical sequences, adapts the layer to the dataset, and sets up the training dataset for efficient model training.

First, the TextVectorization layer encoder is initialized with the specified parameters.

Then, a dataset text_ds is created from the "reviewText" column of the DataFrame. This dataset is batched into groups of 128 samples.

The encoder.adapt(text_ds) line adapts the TextVectorization layer to the text dataset text_ds. This adaptation process builds the vocabulary based on the text data in text_ds and sets up the layer to consistently map words to indices using this vocabulary.

After this adaptation step, the encoder has learned the vocabulary and word-to-index mappings. It doesn't create a new vocabulary for each batch but uses the same vocabulary throughout the training process. When you apply the encoder to any batch of text data using encoder(x), it will consistently map words to indices based on the vocabulary that was learned during the adaptation step.

In the code snippet train_ds = train_ds.map(lambda x, y: (encoder(x), y)), x and y are placeholders for the elements in the dataset train_ds, where x represents the input data (text sequences) and y represents the labels.

In [29]:
max_tokens = 1000
output_sequence_length = 100
pad_to_max_tokens = True

df_train['overall'] = df_train['overall'] - 1
df_train['reviewText'] = df_train['reviewText'].fillna('').astype(str)

encoder = tf.keras.layers.TextVectorization(max_tokens=max_tokens, output_sequence_length=output_sequence_length, pad_to_max_tokens=pad_to_max_tokens)

text_ds = tf.data.Dataset.from_tensor_slices(df_train['reviewText']).batch(128)
encoder.adapt(text_ds)
vocab = np.array(encoder.get_vocabulary())

train_ds = tf.data.Dataset.from_tensor_slices((df_train['reviewText'], df_train['overall'])).batch(128)

train_ds = train_ds.map(lambda x, y: (encoder(x), y))

# Configure the dataset for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [30]:
# Making the test data ready as we did or the training data
df_Xtest['reviewText'] = df_Xtest['reviewText'].fillna('').astype(str)

# Convert the texts to sequences using the already adapted TextVectorization layer
text_test_ds = tf.data.Dataset.from_tensor_slices(df_Xtest['reviewText']).batch(128)
test_ds = text_test_ds.map(lambda x: encoder(x))

# Configure the dataset for performance
AUTOTUNE = tf.data.AUTOTUNE
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [31]:
# Inspect the first few batches of the train_ds
for text_batch, label_batch in train_ds.take(1):  # Adjust .take() for more batches
    for i in range(2):  # Adjust the range to see more or fewer examples
        print("Review:", text_batch.numpy()[i])
        print("Label:", label_batch.numpy()[i])
        print("---")

Review: [ 47  11  10 277 822   5   2 506  13   6   9   1   9 635  69   1   8  12
  10   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]
Label: 3
---
Review: [  3  82   1 111   2 232   1  96   5 352  21  48  21   2 248  96   5 352
   3  19  59   8 211 743  14 908  17   2 248 352 686   5   1  21  48  21
 116 402   3 177  13   6 827   2 149 104  17 220  21 201  21   2  73 216
 459  71   9   7 141  13 577  19  41   2  96  22 120  14   2 104 624 556
 938 622  13  22  49  19   7   1  39 149 104  12   4 131 285   7 112   8
 986   3 177  13   1   2  96  27  11   1]
Label: 4
---


**Input Sequence Length:** You have defined an input_length of 100 for your input sequences. This means that each input sequence is expected to have a maximum length of 100 tokens. If a sequence is shorter than 100 tokens, it will be padded with zeros to reach the specified length.

**Word Embeddings:** Each word in the input sequence is transformed into a dense vector representation (embedding) of size 128 (as you specified). These embeddings are learned during training through a combination of word-to-index mapping and weight multiplication in the Embedding layer.

**LSTM Layer:** The LSTM layer has 128 hidden cells (units). Each hidden cell can capture and maintain information about sequential patterns within the input sequence. These hidden cells are independent of batch size and embedding dimensions, and they contribute to modeling the sequential dependencies in the input data.

**Batch Size:** The batch size determines how many sequences are processed together in parallel during training. The batch size does not affect the architecture of the LSTM layer but rather how many sequences are processed at once before updating the model's weights. It can impact training efficiency and memory usage.

**Vocabulary Size:** The vocabulary size is 999 (the size of the vocabulary). It represents the number of unique words in your dataset that the model is trained on. Each word in the vocabulary has a corresponding embedding vector of size 128, which is learned during training.

**Training:** During training, the model takes input sequences (padded to a length of 100 tokens), converts each token into an embedding vector using the learned vocabulary, and feeds these embeddings into the LSTM layer. The LSTM layer processes the sequences, captures sequential information, and produces an output. The final Dense layer is used for classification tasks.

**Learning:** The model learns the optimal weights for the embeddings, LSTM cells, and the final Dense layer through backpropagation and gradient descent. It optimizes its ability to understand and classify sequences based on the input data and labels, with the LSTM layer helping to model sequential dependencies.

**In summary, the model processes input sequences of a fixed length (100 tokens) using learned embeddings and an LSTM layer to capture sequential patterns. The batch size determines how many sequences are processed together during training, and the vocabulary size and embedding dimensions affect the word representations. The LSTM layer's hidden cells contribute to modeling sequential information within each sequence.**

In [32]:
embedding_dimension = 128
embedding_model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=len(vocab),
                              output_dim=embedding_dimension,
                              input_length=100,
                              name="embedding"),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [33]:
embedding_model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])


embedding_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          128000    
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense_1 (Dense)             (None, 5)                 645       
                                                                 
Total params: 260229 (1016.52 KB)
Trainable params: 260229 (1016.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
embedding_model.fit(train_ds, epochs=1, verbose=1)

242/242 [==============================] - 89s 356ms/step - loss: 1.1276 - accuracy: 0.5892


When you **call embedding_model.predict(test_ds)**, it feeds the test data into the model and generates predictions. The predictions array will contain the predicted probabilities for each class for each sample in your test dataset. Each row in predictions corresponds to a sample, and each column corresponds to a class. So, predictions[i] contains the predicted probabilities for the i-th sample across all classes.

**np.argmax(predictions, axis=1)** calculates the index of the maximum probability for each sample along the columns (axis=1) of the predictions array. This effectively identifies the predicted class for each sample.

plus 1 is added to the result because it seems like your class labels range from 1 to 5. The argmax function returns the index of the maximum value, which is typically a 0-based index. By adding 1, you shift the labels to the range 1 to 5, aligning them with your class labels.

In [35]:
# Make predictions
predictions = embedding_model.predict(test_ds)
# The 'predictions' array will contain the probabilities of each class for each sample

# Convert probabilities to class labels
predicted_labels = np.argmax(predictions, axis=1)+1

# 'predicted_labels' now contains the class label (1 to 5) for each sample in your test dataset
print(predicted_labels)

27/27 [==============================] - 5s 170ms/step
[5 5 5 ... 5 5 4]


Pred for kagge - see other assignments for Colab...

In [ ]:
y_test_hat_pd = pd.DataFrame({
    'Id': list(range(len(predicted_labels))),
    'Predicted': predicted_labels.reshape(-1),
})
y_test_hat_pd.to_csv('y_test_hat.csv', index=False)